# Results and models

#### Setup and load dataset

In [1]:
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import os # I think for cpu
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
from src.functions import *

def evaluate_model(model, x_t, y_t):

    print("\nwith next waypoint prediction")
    result_eval = model.evaluate(x_t,y_t)
    print("MSE: ",result_eval)
    print("---------------------------------------------------")
    print("with autoregressive generation:")

    pred = generate(model ,x_t, traj_n=traj_n).numpy()
    result_gen = np.average((y_t - pred[:,1:,:])**2)
    print("MSE: ",result_gen)
    print("Trajectory metrics:")
    metrics, metrics_h = compute_metrics(y_t.numpy()[:,:,:],pred[:,1:,:])
    return metrics


/home/mghanei3/miniconda3/envs/llm/lib/python3.8/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
2024-04-25 13:54:24.683010: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


### dataset size experiment

In [ ]:
mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("latte_100k_lf", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

test_dataset = tf.data.Dataset.from_tensor_slices((mr.prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)



In [ ]:
# print(X_test[0])
# print(X_test.shape)
import pandas as pd
foo = pd.read_json("data/data.json")
print(foo)
print(foo.iloc[0]['obj_poses'])


In [4]:
# from src.TF4D_mult_features import * # Original - commented out
from src.simple_TF_continuos import *
# model_path = models_folder+"FINAL_dataset_size_aug_fixsteps/"
models_folder = "models/"
model_path = models_folder

# model_names =  [
#                 "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.01-augment:0.h5",
#                 "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.01-augment:1.h5",
#                 "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.1-augment:0.h5",
#                 "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.1-augment:1.h5",
#                 "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.5-augment:0.h5",
#                 "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.5-augment:1.h5",
#                 "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:0.h5",
#                 "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5"]

model_names = ["TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:True-features_n:793-optimizer:adam-norm_layer:True-activation:tanh.h5"]
model_names = ["1st_attempt/TF&num_layers_enc:1&num_layers_dec:6&d_model:256&dff:512&num_heads:8&dropout_rate:0.1&wp_d:2&bs:64&dense_n:512&num_dense:3&concat_emb:True&features_n:793&optimizer:adam&norm_layer:True&activation:tanh.h5"]

models_metrics = {}
for model_name in model_names:

    model_file = model_path+model_name
    # param = file_name2dict(model_file,delimiter="-",show=False)
    # print("======================================================================================================")
    # print("Dataset size: ",str(100.0*float(param["sf"])),"%\t","augmentation: ","ON "if param['augment']==1 else "OFF", "\n")
    # model_tag = "size:" + str(100.0*float(param["sf"]))+"_aug:"+ str(param['augment'])

    model = load_model(model_file, delimiter="-")
    print("model loaded")
    metrics = evaluate_model(model, x_t, y_t)
    print("metric computed")
    models_metrics[model_tag] = metrics


{}
loading weights:  models/1st_attempt/TF&num_layers_enc:1&num_layers_dec:6&d_model:256&dff:512&num_heads:8&dropout_rate:0.1&wp_d:2&bs:64&dense_n:512&num_dense:3&concat_emb:True&features_n:793&optimizer:adam&norm_layer:True&activation:tanh.h5


ValueError: Layer #0 (named "encoder_2" in the current model) was found to correspond to layer encoder in the save file. However the new layer encoder_2 expects 34 weights, but the saved weights have 18 elements.

In [ ]:
print("========== aug 1 ===========\n\t",end="")
for m in ["1k","10k","50k","100k"]:
    print(m, end="\t")
print()
for k in ["mse","mae","dtw","dfd"]:
    print(k, end="\t")
    for n,m in models_metrics.items():
        if "aug:1" in n:
            print(m[k], end="\t")
    print()
print("========= aug 0 ==========")
for k in ["mse","mae","dtw","dfd"]:
    print(k, end="\t")
    for n,m in models_metrics.items():
        if "aug:0" in n:
            print(m[k], end="\t")
    print()

### Num of encoders, decoder and model depth

In [ ]:
from src.TF4D_mult_features import *
model_path = models_folder+"FINAL_enc_dec_depth/"

model_names =  [
"TF-num_layers_enc:1-num_layers_dec:3-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:1-num_layers_dec:3-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:1-num_layers_dec:5-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:2-num_layers_dec:3-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:2-num_layers_dec:3-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:2-num_layers_dec:5-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:2-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5"
]
models_metrics_enc_dec = {}
for model_name in model_names:

    model_file = model_path+model_name
    param = file_name2dict(model_file,delimiter="-",show=False)
    print("======================================================================================================")
    print("num Encoders: ",param["num_layers_enc"],"\tnum Decoders: ",param["num_layers_dec"],"\tDepth: ",param['d_model'])
    model_tag = "enc:"+str(param["num_layers_enc"])+"-dec:"+str(param["num_layers_dec"])+"-d"+str(param['d_model'])

    model = load_model(model_file, delimiter="-")
    metrics = evaluate_model(model, x_t, y_t)
    models_metrics[model_tag] = metrics

In [ ]:
for k in ["mse","mae","dtw","dfd"]:
    print(k, end="\t")
print()
for n,m in models_metrics.items():
    if "dec" in n:
        print(n, end="\t")
        for k in ["mse","mae","dtw","dfd"]:
            print(m[k], end="\t")
        print()



### Naive predictor
Coping initial trajectory (no modifications)

In [ ]:
mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("latte_100k_lf", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

test_dataset = tf.data.Dataset.from_tensor_slices((mr.prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)

In [ ]:
model_tag="naive_predictor"
traj_in = x_t[0][:,MAX_NUM_OBJS+1:,:].numpy()
traj_out = y_t.numpy()
metrics, metrics_h = compute_metrics(traj_out,traj_in)
models_metrics[model_tag] = metrics

### No language

In [ ]:

mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=True)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("latte_100k_lf_cliponly", filter_data = True, base_path=data_folder)
X[:,feature_indices] = 0
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

test_dataset = tf.data.Dataset.from_tensor_slices((mr.prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)

In [ ]:
from src.TF4D_mult_features import *
model_path = models_folder+"FINAL_no_language/"

model_name=  "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:537-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5"
model_tag="no_language"
model_file = model_path+model_name
model = load_model(model_file, delimiter="-")
metrics = evaluate_model(model, x_t, y_t)
models_metrics[model_tag] = metrics

In [ ]:
pred = generate(model ,x_t, traj_n=traj_n).numpy()
data_pred = np.array(data)[indices_test].copy()
for i,d in enumerate(data_pred):
    d["output_traj"] = pred[i]

mr.save_data(data_pred,data_name="test_no_language_100k_latte_f", base_path=data_folder)

### Decoder Only

In [ ]:
mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("latte_100k_lf", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

test_dataset = tf.data.Dataset.from_tensor_slices((mr.prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)

In [ ]:
from src.TF4D_decoder_only import *
model_path = models_folder+"FINAL_decoder_only/"

model_name=  "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:16-bs:16-dense_n:512-num_dense:3-concat_emb:True-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse.h5"

def evaluate_model_dec_only(model, x_t, y_t):

    print("\nwith next waypoint prediction")
    # result_eval = model.evaluate(x_t,y_t)
    # print("MSE: ",result_eval)
    print("---------------------------------------------------")
    pred = model.predict(x_t)
    print("Trajectory metrics:")
    metrics, metrics_h = compute_metrics(y_t.numpy()[:,:,:],pred[:,:,:])
    return metrics

x_t_deconly = (x_t[0][:,MAX_NUM_OBJS:-1,:],x_t[1],x_t[2])
model_tag="decoder_only"
model_file = model_path+model_name
model = load_model(model_file, delimiter="-")
metrics = evaluate_model_dec_only(model,x_t_deconly, y_t)
models_metrics[model_tag] = metrics

In [ ]:
pred = model.predict(x_t_deconly)
data_pred = np.array(data)[indices_test].copy()
for i,d in enumerate(data_pred):
    d["output_traj"] = pred[i]

mr.save_data(data_pred,data_name="test_decoder_only_100k_latte_f", base_path=data_folder)

### clip only

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
from src.functions import *

mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=True)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("latte_100k_lf_cliponly", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

test_dataset = tf.data.Dataset.from_tensor_slices((mr.prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)

In [ ]:
from src.TF4D_mult_features import *
model_path = models_folder+"FINAL_clip_only/"

model_name=  "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:537-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5"
model_tag="decoder_only"
model_file = model_path+model_name
model = load_model(model_file, delimiter="-")
metrics = evaluate_model(model, x_t, y_t)
models_metrics[model_tag] = metrics

### Text Only

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
from src.functions import *

mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("latte_100k_lf_textonly", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

test_dataset = tf.data.Dataset.from_tensor_slices((mr.prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)

In [ ]:
from src.TF4D_mult_features import *
model_path = models_folder+"FINAL_text_only/"

model_name=  "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:537-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5"
model_tag="decoder_only"
model_file = model_path+model_name
model = load_model(model_file, delimiter="-")
metrics = evaluate_model(model, x_t, y_t)
models_metrics[model_tag] = metrics

### Forces interactions

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
from src.functions import *

mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("forces_only_f", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)


def prepare_x(x):
    objs = pad_array(list_to_wp_seq(x[:,obj_poses_indices],d=3),4,axis=-1) # no speed
    trajs = list_to_wp_seq(x[:,traj_indices],d=4)
    #   return np.concatenate([objs,trajs],axis = 1)
    return trajs[:,:-1,:]

test_dataset = tf.data.Dataset.from_tensor_slices((prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)



In [ ]:
from src.TF4D_decoder_only import *

model_path = models_folder+"forces_onl/"
model_name = "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:16-bs:16-dense_n:512-num_dense:3-concat_emb:True-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse.h5"

model_file = model_path+model_name
model = load_model(model_file, delimiter="-")

# metrics = evaluate_model(model, x_t, y_t)
# models_metrics[model_tag] = metrics

In [ ]:
pred = model.predict(x_t)
pred_d = np.array(data)[indices_test]
for i,d in enumerate(pred_d):
    d["forces"] = pred[i]

In [ ]:
%matplotlib qt
indices = np.random.choice(range(len(indices_test)), 3)

plt.close('all')
# pred_t = np.transpose(pred[:,:,:2],[0,2,1])
data_array = np.array(data)[indices_test[indices]]
show_data4D(data_array,plot_output=False)
# data_array = pred_d[indices]
# show_data4D(data_array,plot_output=False)

# show_data4D(data_array,plot_output=False,image_loader=mr.image_loader, color_traj=False, change_img_base=["/home/mirmi/Arthur/dataset/","/home/tum/data/image_dataset/"])
# show_data4D(data_sample,plot_output=False)#,image_loader=mr.image_loader)

# Visualize Results

In [ ]:
from src.TF4D_mult_features import *
model_path = models_folder+"FINAL_dataset_size_aug_fixsteps/"

model_name = "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.5-augment:1.h5"
model_file = model_path+model_name

model = load_model(model_file, delimiter="-")


In [ ]:
pred = generate(model ,x_t, traj_n=traj_n).numpy()
data_pred = np.array(data)[indices_test].copy()
for i,d in enumerate(data_pred):
    d["output_traj"] = pred[i]

mr.save_data(data_pred,data_name="testpred_100k_latte_f", base_path=data_folder)

In [ ]:
# indices = np.random.choice(range(len(indices_test)), 3)
indices = np.arange(3)

In [ ]:
data_pred = mr.load_data("testpred_100k_latte_f", base_path=data_folder)
pred = np.array([d["output_traj"] for d in data_pred])


In [ ]:
%load_ext autoreload
%autoreload 2
from src.functions import *
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# plot best results
th = 0.0005 # maximum mse
good_indices = {"dist":[], "cartesian":[], "speed":[]}
for i,d in enumerate(np.array(data)[indices_test]):
    mse = np.mean((pred[i]-np.array(d["output_traj"]))**2)
    if mse < th and len(d["obj_names"])<4:
        good_indices[d["change_type"]] = good_indices[d["change_type"]] + [i]
indices = [np.random.choice(good_indices[i]) for i in good_indices.keys()]
indices[0] =6276#11699#,11664
# indices[1] =3190
# indices[2] = 14168#11053, 16036, 14168
indices = indices
print(indices)

data_array = np.array(data)[indices_test[indices]]

show_data4D(data_array, pred=pred[indices,:,:],color_traj=False,labels=["Ground Truth","Predicted"],image_loader=mr.image_loader, change_img_base=["/mnt/tumdata/image_dataset/", image_dataset_folder])
